In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import time
from tensorflow.python.keras.datasets import cifar10
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10 - ссылка
from sklearn.preprocessing import OneHotEncoder
input_data = tf.keras.datasets.mnist


TensorFlow 1.x selected.


In [0]:
kernel_height = 3
kernel_width = 3
num_epochs = 25
batch_size = 50
num_loops = int(50000 / batch_size)
learning_rate = 1e-2
num_neurons_full = 512

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def conv_layer(input, shape):
  W = weight_variable(shape)
  b = bias_variable([shape[3]])
  return tf.nn.relu(conv2d(input, W) + b)

def full_layer(input, size):
  in_size = int(input.get_shape()[1])
  W = weight_variable([in_size, size])
  b = bias_variable([size])
  return tf.matmul(input, W)+ b

x = tf.placeholder(tf.float32, shape=[None, 3072])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)

x_image = tf.reshape(x, [-1, 32, 32, 3])

with tf.name_scope('conv_1'):
  conv1 = conv_layer(x_image, shape=[kernel_height, kernel_width, 3, 32])
  conv1_pool = max_pool_2x2(conv1)

with tf.name_scope('conv_2'):
  conv2 = conv_layer(conv1_pool, shape=[kernel_height, kernel_width, 32, 64])
  conv2_pool = max_pool_2x2(conv2)

with tf.name_scope('conv_3'):
  conv3 = conv_layer(conv2_pool, shape=[kernel_height, kernel_width, 64, 128])
  conv3_pool = max_pool_2x2(conv3)
  conv3_flat = tf.reshape(conv3_pool, [-1, 4*4*128])

with tf.name_scope('full_1'):
  full_1 = tf.nn.relu(full_layer(conv3_flat, num_neurons_full))

with tf.name_scope('dropout'):
  full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

with tf.name_scope('activations'):
  y_conv = full_layer(full1_drop, 10)
  tf.summary.scalar('cross_entropy_loss', y_conv)
  (x_train, y_train), (x_test, y_test) = cifar10.load_data()
  
  x_train = x_train / 255
  x_test = x_test / 255

  x_train = x_train.reshape([-1, 3072])
  x_test = x_test.reshape([-1, 3072])

  ohe = OneHotEncoder()
  y_train = ohe.fit_transform(y_train).toarray()
  y_test = ohe.fit_transform(y_test).toarray()

with tf.name_scope('cross'):
  cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_conv,
                                                                            labels=y_))
#SGD
train_step = tf.train.AdagradOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  start_time = time.time()
  
  for epoch in range(num_epochs):
    print("Epoch: {}".format(epoch))
    for i in range(num_loops):
      batch = [x_train[i*batch_size:(i+1)*batch_size],
               y_train[i*batch_size:(i+1)*batch_size]]
      if i % 100 == 0:
        train_accuracy = sess.run(accuracy, feed_dict={x: batch[0],
                                                      y_: batch[1],
                                                      keep_prob: 1.0})
        print("time {}, step {}, training accuracy {}".format(time.time() - start_time,
                                                              i, train_accuracy))
      sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
  
  X = x_test.reshape(10, 1000, 3072)
  Y = y_test.reshape(10, 1000, 10)
  test_accuracy = np.mean([sess.run(accuracy, feed_dict={x: X[i], y_: Y[i], 
                                    keep_prob: 1.0}) for i in range(10)])
  print("test accuracy: {}".format(test_accuracy))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
170500096/170498071 [==============================] - 2s 0us/step
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch: 0
time 0.2627706527709961, step 0, training accuracy 0.10000000149011612
time 11.160618305206299, step 100, training accuracy 0.23999999463558197
time 21.844725847244263, step 200, training accuracy 0.4000000059604645
time 32.56761980056763, step 300, training accuracy 0.30000001192092896
time 44.49670457839966, step 400, training accuracy 0.47999998927116394
time 56.8913848400116, step 500, training accuracy 0.41999998688697815
time 67.5989077091217, step 600, training accuracy 0.5
time 78.32598567008972, step 700, training accuracy 0.5
time 89.04908537864685, step 800, training accuracy 0.5600000023841858
time 99.75270676612854, step 900, training accuracy 0.4399999976158142
Epoch: 